In [ ]:
!pip install keras-segmentation
!pip install neptune-client
#!pip install neptune-contrib

In [ ]:
import os
os.getcwd()
os.chdir('/content/drive/My Drive/SemanticSegmentationV2')
!pwd

In [ ]:
import configparser
config = configparser.ConfigParser()
config.read("./config.cfg")
print()

In [ ]:
import neptune
#from neptunecontrib.monitoring.keras import NeptuneMonitor

# set project
neptune.init(api_token=config["neptune"]["token"],
            project_qualified_name=config["neptune"]["project"])


In [ ]:
# parameters
PARAMS = {'epochs': 100,
          'batch_size': 16,
          'val_batch_size': 16,
          'weights': None,#(1.,1.),
          'input_size': 416,#384,
          'model': 'vgg_unet',
          'patience': 8,
          'metric': 'accuracy',
          'loss': 'categorical_crossentropy',
          'learning_rate': 0.1,
          'decay': None
          }


In [ ]:

# start experiment
experiment = neptune.create_experiment(name=segmentation-keras, params=PARAMS)


In [ ]:
%tensorflow_version 1.x
from keras_segmentation.models.unet import *
from keras_segmentation.models.segnet import *
from keras_segmentation.models.fcn import *
from keras_segmentation.models.pspnet import *
import keras
import os
import tensorflow as tf
import data_loader as dl
import matplotlib.pyplot as pyplot
import pickle
import sys
import time
from datetime import datetime
import math
import numpy as np
import keras.backend as K

In [ ]:
from data_loader import get_image_array, get_segmentation_array,\
    DATA_LOADER_SEED, class_colors, get_pairs_from_paths
from keras_segmentation.predict import predict

def evaluate(model=None, inp_images=None, annotations=None,
             inp_images_dir=None, annotations_dir=None, checkpoints_path=None):

    if inp_images is None:
        assert (inp_images_dir is not None),\
                "Please provide inp_images or inp_images_dir"
        assert (annotations_dir is not None),\
            "Please provide inp_images or inp_images_dir"

        paths = get_pairs_from_paths(inp_images_dir, annotations_dir)
        paths = list(zip(*paths))
        inp_images = list(paths[0])
        annotations = list(paths[1])

    assert type(inp_images) is list
    assert type(annotations) is list

    tp = np.zeros(model.n_classes)
    fp = np.zeros(model.n_classes)
    fn = np.zeros(model.n_classes)
    n_pixels = np.zeros(model.n_classes)
    inter = 0
    union = 0
    licz = 0
    mian = 0
    for inp, ann in zip(inp_images, annotations):
        pr = predict(model, inp)
        gt = get_segmentation_array(ann, model.n_classes,
                                    model.output_width, model.output_height,
                                    no_reshape=True)
        gt = gt.argmax(-1)
        pr = pr.flatten()
        gt = gt.flatten()

        for cl_i in range(model.n_classes):

            tp[cl_i] += np.sum((pr == cl_i) * (gt == cl_i))
            fp[cl_i] += np.sum((pr == cl_i) * ((gt != cl_i)))
            fn[cl_i] += np.sum((pr != cl_i) * ((gt == cl_i)))
            n_pixels[cl_i] += np.sum(gt == cl_i)
        #new
        temp = np.sum(pr*gt)
        union += np.sum(pr+gt)-temp
        inter += temp
        licz += np.sum((pr == gt))
        mian += gt.size
    my_iou = (inter + 0.000000000001) / (union + 0.000000000001)
    my_acc = (licz + 0.000000000001) / (mian + 0.000000000001)
    cl_wise_score = tp / (tp + fp + fn + 0.000000000001)
    n_pixels_norm = n_pixels / np.sum(n_pixels)
    frequency_weighted_IU = np.sum(cl_wise_score*n_pixels_norm)
    mean_IU = np.mean(cl_wise_score)

    return {
        "frequency_weighted_IU": frequency_weighted_IU,
        "mean_IU": mean_IU,
        "class_wise_IU": cl_wise_score,
        'my_iou': my_iou,
        'my_acc': my_acc
    }


In [ ]:
def get_model_by_name(name):
    n_classes = 2
    if name == "vgg_unet":
        return vgg_unet(n_classes, input_height=input_height,  input_width=input_width)
    elif name == "resnet50_unet":
        return resnet50_unet(n_classes, input_height=input_height,  input_width=input_width)
    elif name == "unet":
        return unet(n_classes, input_height=input_height,  input_width=input_width)
    elif name == "segnet":
        return segnet(n_classes, input_height=input_height,  input_width=input_width)
    elif name == "vgg_segnet":
        return vgg_segnet(n_classes, input_height=input_height,  input_width=input_width)
    elif name == "resnet50_segnet":
        return resnet50_segnet(n_classes, input_height=input_height,  input_width=input_width)
    elif name == "fcn_8":
        return fcn_8(n_classes, input_height=input_height,  input_width=input_width)
    elif name == "fcn_32":
        return fcn_32(n_classes, input_height=input_height,  input_width=input_width)
    elif name == "fcn_8_vgg":
        return fcn_8_vgg(n_classes, input_height=input_height,  input_width=input_width)
    elif name == "fcn_32_vgg":
        return fcn_32_vgg(n_classes, input_height=input_height,  input_width=input_width)
    elif name == "fcn_8_resnet50":
        return fcn_8_resnet50(n_classes, input_height=input_height,  input_width=input_width)
    elif name == "fcn_32_resnet50":
        return fcn_32_resnet50(n_classes, input_height=input_height,  input_width=input_width)
    elif name == "pspnet":
        return pspnet(n_classes, input_height=input_height,  input_width=input_width)
    elif name == "vgg_pspnet":
        return vgg_pspnet(n_classes, input_height=input_height,  input_width=input_width)
    elif name == "resnet50_pspnet":
        return resnet50_pspnet(n_classes, input_height=input_height,  input_width=input_width)

In [ ]:




def iou(y_true, y_pred):
    #input shape [n,43264,2]
    y_true = y_true[:,:,1]
    y_pred = y_pred[:,:,1]
    inter = K.sum(K.sum(y_true*y_pred, axis=1), axis=0)
    union = K.sum(K.sum(y_true+y_pred, axis=1), axis=0) - inter
    return (inter + K.epsilon()) / (union + K.epsilon())

def iou_loss(y_true, y_pred):
    #input shape [n,43264,2]
    y_true = y_true[:,:,1]
    y_pred = y_pred[:,:,1]
    inter = K.sum(y_true*y_pred, axis=1)
    union = K.sum(y_true+y_pred, axis=1) - inter
    iou_val = (inter + K.epsilon()) / (union + K.epsilon())
    return -K.log(iou_val)

# pathes constants
train_img_dir = os.path.join("dataset","x_train")
train_seg_dir = os.path.join("dataset","y_train")
test_img_dir = os.path.join("dataset","x_test")
test_seg_dir = os.path.join("dataset","y_test")

# constants
batch_size = PARAMS['batch_size']
steps_per_epoch = int(math.ceil(len(os.listdir(train_img_dir))/batch_size))
val_batch_size = PARAMS['val_batch_size']
val_steps_per_epoch = int(math.ceil(len(os.listdir(test_img_dir))/val_batch_size))

# define model
n_classes=2
input_height=PARAMS['input_size']
input_width=PARAMS['input_size']
model = get_model_by_name(PARAMS['model'])
dir = PARAMS['model']+"_"+datetime.now().strftime("%d%m%Y_%H%M%S")
os.mkdir(os.path.join(".","dump",dir))
output_height = model.output_height
output_width = model.output_width
print(">>>")
print(input_height)
print(input_width)
print(output_height)
print(output_width)
print(">>>")
model.summary()

if PARAMS['metric'] == 'iou':
    metrics = [iou]
elif PARAMS['metric'] == 'accuracy':
    metrics = ['accuracy']

if PARAMS['loss'] == 'categorical_crossentropy':
    loss = 'categorical_crossentropy'
elif PARAMS['loss'] == 'iou_loss':
    loss = iou_loss
#if PARAMS['weights'] is None:
learning_rate = PARAMS['learning_rate']
if PARAMS['decay'] is None:
  optimizer = keras.optimizers.Adadelta(learning_rate = PARAMS['learning_rate'])
else:
  optimizer = keras.optimizers.Adadelta(learning_rate = PARAMS['learning_rate'], rho=PARAMS['decay'])
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
#else:
#    pass#model.compile(optimizer="adadelta", loss=loss, metrics=metrics, sample_weight_mode="temporal")

# create data generators
weights = PARAMS['weights']
#train_gen = dl.image_segmentation_generator(train_img_dir, train_seg_dir, batch_size, n_classes, input_height, input_width, output_height, output_width, weights)
train_gen = dl.DataGenerator(train_img_dir, train_seg_dir, batch_size, input_height, input_width, output_height, output_width)#, weights)

#val_gen = dl.image_segmentation_generator(test_img_dir, test_seg_dir, val_batch_size, n_classes, input_height, input_width, output_height, output_width, weights)
val_gen = dl.DataGenerator(test_img_dir, test_seg_dir, val_batch_size, input_height, input_width, output_height, output_width)#, weights)



In [ ]:

mc = keras.callbacks.ModelCheckpoint(os.path.join(".","dump",dir,"weights.best.hdf5"), monitor='val_'+PARAMS['metric'], verbose=1, save_best_only=True, mode='max')
es = keras.callbacks.EarlyStopping(monitor='val_'+PARAMS['metric'], mode='max', verbose=1, patience=PARAMS['patience'])
callbacks = [mc, es]#, NeptuneMonitor()]

# fit model

history = model.fit_generator(train_gen, validation_data=val_gen,
                            epochs=PARAMS['epochs'], callbacks=callbacks, use_multiprocessing=False)
# print training info
print("###")
#print(th.times)
pyplot.plot(history.history[PARAMS['metric']], label='train')
pyplot.plot(history.history['val_'+PARAMS['metric']], label='test')
pyplot.legend()
pyplot.show()
pickle.dump(history.history, open(os.path.join(".","dump",dir,"history.obj"), 'wb'))



In [ ]:
# evaluate the model
weights_path = os.path.join(".","dump",dir,"weights.best.hdf5")
print("Weights path:")
print(weights_path)
model.load_weights(weights_path)


In [ ]:
my_eval_dict = evaluate(model, inp_images_dir=os.path.normpath("dataset/X_test/")  , annotations_dir=os.path.normpath("dataset/Y_test/" ) )
print(my_eval_dict)

In [ ]:
neptune.log_metric('eval_iou', my_eval_dict['my_iou'])
neptune.log_metric('eval_accuracy', my_eval_dict['my_acc'])

In [ ]:
for val in history.history['accuracy']:
  neptune.log_metric('hist_accuracy', val)


In [ ]:
for val in history.history['loss']:
  neptune.log_metric('hist_loss', val)


In [ ]:
for val in history.history['val_accuracy']:
  neptune.log_metric('hist_val_accuracy', val)


In [ ]:
for val in history.history['val_loss']:
  neptune.log_metric('hist_val_loss', val)


In [ ]:
neptune.state = 'succeeded'

In [ ]:
print(latest)
model.load_weights(latest)
val_gen = image_segmentation_generator(test_img_dir, test_seg_dir, 1,
                                 n_classes, input_height, input_width,
                                 output_height, output_width)
rgb = next(val_gen)
print(np.array(rgb[1]))
#pyplot.imshow(val_gen.)

#print(model.evaluate_segmentation( inp_images_dir=os.path.normpath("dataset/test_images/")  , annotations_dir=os.path.normpath("dataset/test_segmentation/" ) ))

In [ ]:
import importlib
importlib.reload(image_segmentation_generator)

In [ ]:
from keras_segmentation.predict import predict_multiple
import os

model_dir = os.path.join("best","unet_11122020_113849")
model = get_model_by_name("unet")
weights = os.path.join(model_dir,"weights.best.hdf5")
model.load_weights(weights)
test_img_dir = os.path.join("dataset","X_test")
test_seg_dir = os.path.join("dataset","Y_test")
out_dir = os.path.join(model_dir,"out")
os.mkdir(out_dir)

colors = [(0,0,0),(255,255,255)]
predict_multiple(model=model, inps=None, inp_dir=test_img_dir, out_dir=out_dir,
                     checkpoints_path=None, overlay_img=False,
                     class_names=None, show_legends=False, colors=colors,
                     prediction_width=None, prediction_height=None)

In [ ]:
import os
import cv2

test_seg_dir = os.path.join("dataset","Y_test")
out_dir = os.path.join("best","ground_truth")
if not os.path.isdir(out_dir):
  os.mkdir(out_dir)
img_files = os.listdir(test_seg_dir)
for img_file in img_files:
  img = cv2.imread(os.path.join(test_seg_dir,img_file),0)
  thr, img = cv2.threshold(img,0,255,cv2.THRESH_BINARY)
  print(img)
  cv2.imwrite(os.path.join(out_dir,img_file.split(".")[0]+".jpg"),img)
